## 🔮 Deep Learning in Practice


<img src="https://i.imgur.com/0OQVA5H.png">

Most of what we have covered applies to Keras 2.x where Keras is considered as a submodule of TensorFlow. Keras 3.0 which has been released just a few months ago has improved in few dimensions:

<img src="https://i.imgur.com/qq2BDBT.png">

It's no longer dependent on TensorFlow. It could also be used with PyTorch or Jax!

How? Remeber that we argued before that with `torch` or `tensorflow` only, we can build any architecture we want (since layers in `tf.keras.layers` or `torch.nn` are just mathematical operations that can be implemented with the base functionality (tensor processing library) which also includes GPU compuation and automatic differentiation).

- Thus, with keras 3.0, when an archiecture is defined, it can be used as a TensorFlow, PyTorch or Keras model by setting an environment varianle `os.environ['backend']`.

- If `os.environ['backend'] = 'torch'` then it will use implementations of the layers in the architecture using `torch` and if it's `os.environ['backend'] = 'tensorflow'` then it will use tensorflow implementations and so on.



In [ ]:
!pip install --upgrade keras                # Get Keras 3.0
import keras                                # Use it!

 Most if not all what we had before remains the same otherwise, just that lines like `tf.keras.Model` or `tf.keras.layers.Dense` will become `keras.Model` and `keras.layers.Dense` respectively.

Let's try this out:

In [1]:
import os; os.environ['backend'] = 'torch'

#### Define Keras Model

In [2]:
import keras
from tqdm.keras import TqdmCallback
from keras.layers import Dense, LeakyReLU
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy


class NeuralNet(keras.Model):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.model = keras.Sequential([
            Dense(256),
            LeakyReLU(negative_slope=1e-2),
            Dense(128),
            LeakyReLU(negative_slope=1e-2),
            Dense(1)
        ])

    def call(self, inputs):
        return self.model(inputs)

#### Prepare Data

In [3]:
from sklearn.datasets import make_moons
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

x_data, y_data = make_moons(n_samples=700, noise=0.1, random_state=42)

# make tensor dataset
x_data_tensor = torch.tensor(x_data, dtype=torch.float32)
y_data_tensor = torch.tensor(y_data, dtype=torch.float32).unsqueeze(1)

dataset = TensorDataset(x_data_tensor, y_data_tensor)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

#### Build and Train Keras Model with PyTorch Backend

In [4]:
model = NeuralNet()
model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss=BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'],
            run_eagerly=False
            )

model.fit(train_loader, epochs=50, verbose=False, callbacks=[TqdmCallback()])

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

#### Let's try it out:

In [5]:
model.evaluate(val_loader, return_dict=True)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0102  


{'accuracy': 1.0, 'loss': 0.010292550548911095}

As an exercise, wrap the dataset in TensorFlow `Dataset` object and use a TensorFlow backend.

If we wish, we can even use Keras model with custom PyTorch loop (similar to what we did before with TensorFlow). For examples, [see this](https://keras.io/guides/writing_a_custom_training_loop_in_torch/) and [this](https://keras.io/guides/custom_train_step_in_torch/). PyTorch optimizers/losses are even allowed.

### 🤔 So what's Keras.ops?

It (supposedly) has all [tensor operations](https://keras.io/api/ops/) found in the core of packages such as PyTorch, TensorFlow, Jax. The implementation of the operation is decided based on `os.environ['backend]`.

Hence, it helps you write deep learning code that will work for with all deep learning frameworks!

For instance, we earlier made the following implementation for a linear layer:

```python
class Linear(tf.keras.layers.Layer):
    def __init__(self, input_dim=20, output_dim=32):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, output_dim),trainable=True)
        self.b = self.add_weight(shape=(output_dim,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    

layer = Linear(20, 32)                               # a layer with 32 neurons; input size will be inferred
inputs = tf.random.uniform(shape=(16, 20))          # batch of 16 examples, each of dimensionality 20
outputs = layer(inputs)
outputs.shape
```

This layer is not generic, if we use it in a model then we can't switch to PyTorch implementation with `os.environ['backend'] = "torch"` as specific TensorFlow code is written. To make it generic, we use the generic `matmul` found in `keras.ops`:

In [9]:
class Linear(keras.layers.Layer):
    def __init__(self, input_dim=32, output_dim=20):
        super().__init__()
        self.w = self.add_weight(
            shape=(input_dim, output_dim),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(shape=(output_dim,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return keras.ops.matmul(inputs, self.w) + self.b
    
layer = Linear(20, 32)                              
inputs = torch.rand(16, 20)          # batch of 16 examples, each of dimensionality 20
outputs = layer(inputs)
outputs.shape

TensorShape([16, 32])

If we switch to TensorFlow and send a TF tensorflow object, it will still work:

In [10]:
import os; os.environ['backend'] = 'tensorflow'

In [12]:
import tensorflow as tf

layer = Linear(20, 32)                               # a layer with 32 neurons; input size will be inferred
inputs = tf.random.uniform(shape=(16, 20))          # batch of 16 examples, each of dimensionality 20
outputs = layer(inputs)
outputs.shape

TensorShape([16, 32])

**This is fantastic because it allows:**
- Using PyTorch via a high-level API (e.g., not writing training loop, using metrics or callbacks, etc.)
   - There is an independent package called `PyTorch Lightning` built on top of PyTorch that you can check out. However, it of course doesn't support multiple backends and not a lot of people like it.

- Developing in your favorite framework (e.g., PyTorch) then deploying on another framework where it's easier (e.g., TensorFlow)

Prior to this, if a model is saved in the ONNX format (easy to find out how for any framework) then it can (theoretically) be loaded on any other framework, if the operations used in the architecture don't go beyond what ONNX supports.

ONNX stands for `Open Neural Network Exchange`.

<img src="https://media2.giphy.com/media/26u4lOMA8JKSnL9Uk/giphy.gif?cid=6c09b952mz8j6vf96mg8afz545e2a13zfmmtt46g1r85h6jc&ep=v1_gifs_search&rid=giphy.gif&ct=g">

Now that we understand TensorFlow and Keras in sufficient depth. Let's revisit the applications we did before with PyTorch but this Keras/TF instead.